In [4]:
import sys
import os
import mmh3
import numpy as np
import itertools
import collections
import pandas as pd
import more_itertools
from itertools import tee
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
ROOT_DIR = os.path.abspath(os.curdir)
#################### Utilities ######################
#hashes a list of strings
def listhash(l,seed):
	val = 0
	for e in l:
		val = val ^ mmh3.hash(e, seed)
	return val 

In [11]:
#import our dataset and then put them into the dataframe 
# data = pd.read_csv(ROOT_DIR+"\\data\\articles1.csv")
# data_content = data["content"] #we are only interested in the content in each news
########################
docs = {} #dictionary mapping document id to document contents

data = pd.read_csv(ROOT_DIR+"\\data\\articles1.csv")

data_content = data[:50000]["content"]
# docs = list(zip(data_content.tolist())) ## convert to tuple for starmap to work basically getting ['dfdf', 'dddd', 'sss'] => [('dfdf'), ('dddd'), ('sss')]
# print(docs[2])


In [ ]:
#import the document of stopwords 
stop = set(stopwords.words('english'))
def clean(text):
    """
    This function takes as input a text on which several 
    NLTK algorithms will be applied in order to preprocess it
    """
    
    tokens = word_tokenize(text)
    # Remove the punctuations, lower tokens and remove stopword
    tokens = [word.lower() for word in tokens if word not in stop and len(word) > 2]
   
    return tokens

In [ ]:
docs1={}
for i in data_content.index:  
    cleaned = clean(data_content[i])   
    docs1[i] = str([' '.join(cleaned)])

In [ ]:
def hashed_lst_shingles2(q, doc):
    
    doc = doc.split(" ")
    lst_shingles=[]

    lst_shingles = [doc[i:i+q] for i in range(0, len(doc), q-2)] # create list of shingles of length q

    lst_shingles = [x for x in lst_shingles if len(x)==q] # remove shingles with length < q

    return lst_shingles 

In [ ]:
''' Extension of standard pairwise function to 3-pairwise from the py standard lib'''
def three_pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    # three_pairwise('ABCDEFG') --> ABC BCD CDE DEF EFG FG
    a, b, c = tee(iterable, 3) ## Create three iterators
    next(b, None) ## Advance the second
    next(c, None) ## Advance the third
    next(c, None) ## Advance the third once again. This ensures the third iterator starts at the third element and that we can create the 3-tuple
    return zip(a, b, c) ### Zip everything (1 elem, 2 elem, and 3 elem) concurrently.

In [ ]:
docs_clean = {}
count = 1
for i,k in docs1.items():
  docs_clean["m{0}".format(count)] = hashed_lst_shingles2(3,k)
  count +=1 

In [ ]:
num_perm = 100
min_dict1 = {}
count3 = 1
for val in tqdm(docs_clean.values()):
    m = MinHash(num_perm=num_perm)
    for shingle in val:    
      a= []
      for i in shingle:
        i.encode('utf8')
        a.append(i)
      data1 = str(['_'.join(a)])

      m.update(data1.encode('utf8'))
    min_dict1["m{}".format(count3)] = m
    count3+=1

In [ ]:
    
lsh = MinHashLSH(threshold=0.54, num_perm=num_perm)
for key in tqdm(min_dict1.keys()):
    lsh.insert(key,min_dict1[key]) # insert minhash data structure

In [60]:
import random
def ror(n,rotations=1,width=32):
    return (n >> rotations)|(n << (width - rotations)) & 0xFFFFFFFF
    # return (2**width-1)&(n>>rotations|n<<(width-rotations))

def gen_hashval(n):
    return ror(n)^random.randint(0, 2**63-1)

''' Extension of standard pairwise function to 3-pairwise from the py standard lib'''
def three_pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    # three_pairwise('ABCDEFG') --> ABC BCD CDE DEF EFG FG
    a, b, c = tee(iterable, 3) ## Create three iterators
    next(b, None) ## Advance the second
    next(c, None) ## Advance the third
    next(c, None) ## Advance the third once again. This ensures the third iterator starts at the third element and that we can create the 3-tuple
    return zip(a, b, c) ### Zip everything (1 elem, 2 elem, and 3 elem) concurrently.

def hash_docs(doc, k=1000):
    # three_tup=zip
    lst_shingles_h=[]
    # for doc in docs:
    seed=1
    three_tup=three_pairwise(doc.split(" "))

    three_tup=list(three_tup)

    three_tup=[elem for elem in three_tup if '' not in elem]
    
    # for t in three_tup:
    #     if '' or ' ' in t:
    #         print("single q", t)
    #     if " " or "" in t:
    #         print("double ", t)

    # print(set(three_tup))

    # seeds = [seed+i for i in range(0,k)]
    # ret_lst=[]
    # for s in seeds:  
    #     ret_lst.append(np.array(min([listhash(shingle, s) for shingle in three_tup], default=0)))  
    # lens=[]
    # my_lst=[]
    # for shingle in three_tup:
    #     my_lst.append(listhash(shingle, seed))
    #     lens.append(len(my_lst))
    # ret_lst.append(np.array(sorted(my_lst)[:k]))
    # print("min len", min(lens))
    # ret_lst.append(np.array(sorted([listhash(shingle, seed) for shingle in three_tup])[:k]))
    # print("min leng", min(lens))
    
    # s_lst = sorted([listhash(shingle, seed) for shingle in three_tup])
    # minhash_val = [listhash(shingle, seed) for shingle in three_tup]
    '''compute other hash values based on this'''
    # s_lst=[min([listhash(shingle, s) for shingle in three_tup], default=0) for s in seeds]
    
    # s_lst = [gen_hashval(minhash_val) for _ in range(k-1)]
    # s_lst.append(minhash_val)
    # s_lst=[]
    # s_lst.append(min(minhash_val, default=0))
    # for _ in range(k-1):
    #     s_lst.append(min([gen_hashval(minval) for minval in minhash_val], default=0))

    

    # return np.array(s_lst)##[:k])#, len(s_lst)
    return np.array(sorted([listhash(shingle, seed) for shingle in three_tup]))

''' Returns document as a list of hashes'''
'''
Creates shingles of size q, removing shingles of size < q. Removes duplicates and hashes the result.
'''
def hashed_lst_shingles(my_docs):


    hashed_docs = itertools.starmap(hash_docs, my_docs)
    return hashed_docs

# doc = "You and me, we made a vow. For better or for worse. I can't believe you let me down"
# lst_shnigles = hashed_lst_shingles(3, doc)
# print(lst_shnigles)
# [['You', 'and', 'me,'], ['and', 'me,', 'we'], ['me,', 'we', 'made'], ['we', 'made', 'a'], ['made', 'a', 'vow.'], ['a', 'vow.', 'For'], ['vow.', 'For', 'better'], ['For', 'better', 'or'], ['better', 'or', 'for'], ['or', 'for', 'worse.'], ['for', 'worse.', 'I'], ['worse.', 'I', "can't"], ['I', "can't", 'believe'], ["can't", 'believe', 'you'], ['believe', 'you', 'let'], ['you', 'let', 'me'], ['let', 'me', 'down']]

In [61]:
# s_lst=[1 for _ in range(100)]
# a_lst=[2 for _ in range(100)]
# res=[s_lst, a_lst]
# np.stack(res).shape

### MinHashing implementation

In [67]:
# test_docs=list(zip(["You and me, we made a vow. For better or for worse. I can't believe you let me down",
# "Time, space and state. Equal everything explanable.",
# "You and me, we made a vow. For better or for worse. I can't believe you let me down"]))
hashed_shingles=hashed_lst_shingles(docs)
# print(hash)
hashed_shingles = list(hashed_shingles)
hashed_shingles=[lst for lst in hashed_shingles if len(lst)>100]
# hashed_shingles
sig_m=np.array(hashed_shingles).T #5000x100 -> 100x5000
print(sig_m.shape)


(47521,)


C:\Users\hasee\AppData\Local\Temp\ipykernel_4780\799380711.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sig_m=np.array(hashed_shingles).T #5000x100 -> 100x5000


In [68]:
lens=min([len(x) for x in sig_m])
print(lens)
num_docs=sig_m.shape[0]
for i in range(num_docs):
    sig_m[i]=sig_m[i][:lens]
sig_m=np.stack(sig_m).T
sig_m.shape


101


(101, 47521)

In [64]:
# my_s=[]
# for i in range(len(hashed_shingles)):
#     my_s.append(hashed_shingles[i][1])
# min(my_s)
# # min(my_s)

### LSH implementation

In [69]:
''' Jaccard similarity'''
def jaccard(s1, s2):
    return len(s1 & s2) / len(s1 | s2)

''' Implementation of LSH, dividing signature matrix into b band with r rows each'''
def LSH(sig_m, b, r):
    b=20
    r=5
    #b*r=num_hash_funcs

    sim_hashes=[]
    start=0
    for i in range(b):
        sim_hashes.append([listhash(col, seed=i) for col in sig_m[start:start+r,:].T])
        start=i+r

    return sim_hashes

''' Find candidate pairs by checking to see if the hashes match.
Then we check to see that the Jaccard similarity b/w each pair of docs is atleast t. If so, we consider it a candidate pair otherwise not '''
def get_cand_pairs(sim_hashes, t):
    cand_pairs=set()
    for L in sim_hashes:
        dups = collections.defaultdict(list)
        for i, e in enumerate(L):
            dups[e].append(i)
        for _, v in sorted(dups.items()):
            if len(v) >= 2:
                cand_pairs.add(tuple(v))
    cand_pairs=list(cand_pairs)
    filtered_cand_pairs = [pair for pair in cand_pairs if (jaccard(set(sig_m[:, pair[0]]), set(sig_m[:, pair[1]])) > t)]
    return filtered_cand_pairs
   

In [70]:
b=20
r=5
sim_hashes = LSH(sig_m, b=20, r=5)
pairs=get_cand_pairs(sim_hashes, t=(1/b)**(1/r))
len(pairs)
# pairs

31

In [71]:
sig_m.shape

(101, 47521)

In [72]:
pairs

[(47381, 47422),
 (42957, 42965),
 (43171, 43321),
 (45465, 45663),
 (41980, 42274),
 (15705, 24350),
 (44545, 44740),
 (15236, 17272),
 (47032, 47046),
 (41620, 41633),
 (40643, 40763),
 (44249, 44435),
 (47384, 47518),
 (38982, 38995),
 (43546, 43845, 44135),
 (38765, 38816),
 (45476, 45664),
 (19563, 29213),
 (45790, 45997),
 (41101, 41187),
 (46647, 46703),
 (41786, 47187),
 (10977, 12007),
 (8035, 30924),
 (16620, 44519),
 (46331, 46346, 46704),
 (41731, 41980, 42274),
 (46330, 46345),
 (41619, 41632, 41979, 47031),
 (43546, 44135),
 (47380, 47421)]

As can be seen using LSH, we find that documents 5 and 6 are similar.

In [ ]:
docs[41116]

(' (CNN) Film director David Lynch has confirmed he will no longer direct the revival of ”Twin Peaks”    a cult 1990s television show that was set to return in 2016. The offbeat TV series, created by Lynch and Mark Frost, featured a quirky FBI agent who went to the Pacific Northwest town of Twin Peaks to investigate the mysterious murder of a high school girl named Laura Palmer. The groundbreaking series is considered one of the most influential shows in television history. Lynch broke the news about his departure in a series of tweets, saying that the show’s third season will continue without him. He said he felt the network was not offering enough money to produce the show ”the way it needed to be done.” Lynch also wrote that he had personally called the actors over the weekend to let them know he would no longer be directing. Showtime Network, which will air the   comeback, released a statement saying they were ”saddened” by Lynch’s decision. ”We were saddened to read David Lynch’s 

In [ ]:
docs[41048]

(' (CNN) Welcome to the first of April. It’s like Super Bowl Sunday for dad jokes, a day when we can all wake up, fire up the internet, and sigh a collective groan at the barrage of   pranks, tall tales and PR stunts conjured up by desperate media companies and junior brand creatives across the world. We’ll be here as long as we can take it today,   ourselves into oblivion as we catalog the good, the bad and the ugly of April Fools’ Day 2015. If you see anything worth mentioning, let us know in the comments.   eats his way across Google Maps, Google, the merry prankster of    corporate Web behemoths, has announced that you can now play   on Google Maps. Open a map in your browser, click the blue and black square in the bottom left of your screen, and before you know it, it’ll be lunchtime. Your UberBOAT is arriving, Why dart through the streets of your city in a Prius when you can take to the high seas? The dream has (very nearly) (not really) become reality in Thailand, where Uber wil